# Trying stuff

In [6]:
import dhlab as dh
from IPython.display import HTML, Markdown
import os
import sqlite3
import pandas as pd

You may add your own css for displaying the notebook

In [7]:
dh.css("medium.css")

In [8]:
!ls /mnt/disk2/NB-ngram-assoc/

association-notes.html	 galakser.py		   stop_words_basex_ft.txt
avis-bigram-all-row.db	 galaxy2gexf.py		   top100.csv
avis-bigram-one-row.db	 galaxy2graphml.py	   top100first.txt
avis_entropy_all.csv	 kun-info-viten.csv	   trigram-all-row.db
avis_entropy.csv	 kunnskap-viten-info.csv   trigram-entiteter-avis.csv
avis-trigram-all-row.db  ngram_parser.ipynb	   trigram-entiteter-avis.db
avis-trigram-one-row.db  ngram_viewer_ndvd3.ipynb  trigram-one-row.db
avis-unigram-all-row.db  paratakse.txt		   unigram-all-row.db
avis-unigram-one-row.db  parse_nor.py		   unigram-one-row.db
bigram-all-row.db	 parse.py		   untitled
bigram-one-row.db	 parse.py~		   updates.sql
d3force.py		 __pycache__
David.ipynb		 sqlite3_query_ft.ipynb


In [9]:
ngram_root = "/mnt/disk2/NB-ngram-assoc/"

In [10]:
with open("/mnt/disk2/NB-ngram-assoc/association-notes.html") as f:
    x = f.read()

In [11]:
uni = "/mnt/disk2/NB-ngram-assoc/unigram-one-row.db"
bi = "/mnt/disk2/NB-ngram-assoc/bigram-one-row.db"
tri = "/mnt/disk2/NB-ngram-assoc/trigram-one-row.db"

In [12]:
def query(db, sql, params=()):
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        res = cur.execute(sql, params).fetchall()
    return res

In [13]:
query(tri, "select * from sqlite_master")

[('table',
  'trigram',
  'trigram',
  2,
  'CREATE TABLE trigram (freq int, lang varchar, first varchar, second varchar, third varchar, json text, assoc float)'),
 ('index',
  '_lfstf_',
  'trigram',
  7345631,
  'CREATE INDEX _lfstf_ on trigram(lang,first,second,third,freq)'),
 ('index',
  '_lf_',
  'trigram',
  8176254,
  'CREATE INDEX _lf_ on trigram(lang,freq)'),
 ('index',
  '_lstf_',
  'trigram',
  8581773,
  'CREATE INDEX _lstf_ on trigram (lang,second,third,freq)'),
 ('index',
  '_ltf_',
  'trigram',
  9271437,
  'CREATE INDEX _ltf_ on trigram (lang,third,freq)'),
 ('index',
  '_lftf_',
  'trigram',
  9816345,
  'CREATE INDEX _lftf_ on trigram (lang,first,third,freq)'),
 ('index',
  '_lfsta_',
  'trigram',
  11071163,
  'CREATE INDEX _lfsta_ on trigram (lang, first, second, third, assoc)'),
 ('index',
  '_la_',
  'trigram',
  12091504,
  'CREATE INDEX _la_ on trigram (lang, assoc)'),
 ('index',
  '_lsta_',
  'trigram',
  12684194,
  'CREATE INDEX _lsta_ on trigram (lang, secon

In [14]:
def search(expr, lang = 'nob', limit = 10):
    mapping = {0:"first", 1:"second", 2:"third"}
    func_map = {0:unigrams, 1:bigrams, 2:trigrams}
    kwargs = dict()
    for i,e in enumerate([x.strip() for x in expr.split()]):
        if i > 2:
            i = 2
            break
        if e != "*":
            kwargs[mapping.get(i)] = e
    return func_map[i](lang = lang, limit=limit, **kwargs)
            
        


def trigrams(first=None, second=None, third=None, limit = 10, lang='nob', order='assoc'):
    a = locals()
    lang = a['lang']
    limit = a['limit']
    order = a['order']
    qs = [f"{k} = '{a[k]}'" for k in a if a[k] is not None and k in ['first','second', 'third']]
    q = ' and '.join(qs)
    if q != "":
        q = " and " + q
    res = query(tri, f"""
        select first, second, third, freq, assoc 
        from trigram 
        where 
            lang = '{lang}' 
            {q} 
        order by {order} 
        desc 
        limit {limit}""")
    return pd.DataFrame(res, columns="first second third freq assoc".split())

def bigrams(first=None, second=None, limit = 10, lang='nob', order='pmi'):
    a = locals()
    lang = a['lang']
    limit = a['limit']
    order = a['order']
    qs = [f"{k} = '{a[k]}'" for k in a if a[k] is not None and k in ['first','second', 'third']]
    q = ' and '.join(qs)
    if q != "":
        q = " and " + q
    res = query(bi, f"""
        select first, second, freq, pmi 
        from bigram 
        where 
            lang = '{lang}' 
            {q} 
        order by {order} 
        desc 
        limit {limit}""")
    return pd.DataFrame(res, columns="first second freq pmi".split())

def unigrams(first=None, limit = 10, lang='nob', order='freq'):
    a = locals()
    lang = a['lang']
    limit = a['limit']
    order = a['order']
    qs = [f"{k} = '{a[k]}'" for k in a if a[k] is not None and k in ['first','second', 'third']]
    q = ' and '.join(qs)
    if q != "":
        q = " and " + q
    res = query(uni, f"""
        select first, freq 
        from unigram 
        where 
            lang = '{lang}' 
            {q} 
        order by {order} 
        desc 
        limit {limit}""")
    return pd.DataFrame(res, columns="first freq".split())

In [31]:
bigrams(second='gående', order='pmi',  limit=50).apply(lambda x: x['first'][-1] != '-')

,first,second,freq,pmi
0,videre-,gående,567,54.450370
1,N-S-,gående,134,49.184564
2,kom,gående,20396,48.752277
3,nord-sør-,gående,239,47.886380
4,Ø-V-,gående,75,44.571862
5,øst-vest-,gående,133,39.628158
6,fore-,gående,229,36.828881
7,NV-SØ-,gående,36,36.402415
8,NØ-SV-,gående,31,34.481320
9,holdes,gående,885,33.983719


In [111]:
trigrams(third='persille',  limit=50)

,first,second,third,freq,assoc
0,spiseskefuld,finhakket,persille,23,26.271626
1,spiseske,finhakket,persille,230,25.876339
2,spiseskeer,finhakket,persille,23,23.999892
3,teske,finhakket,persille,37,23.609906
4,ss,finklippet,persille,84,23.030260
5,Dryss,hakket,persille,163,22.814763
6,spiseske,hakket,persille,28,22.761524
7,spiseskje,finhakket,persille,30,22.616359
8,spiseskje,hakket,persille,66,22.395876
9,ss,finhakket,persille,723,22.356944


In [95]:
unigrams(  limit=50)

,first,freq
0,.,579382440
1,",",471880905
2,og,259886630
3,i,225005888
4,som,128038992
5,det,127565461
6,er,123343894
7,av,122090298
8,til,121192030
9,en,116360555


In [77]:
query(tri, 'CREATE INDEX _lsa_ on trigram (lang,second,assoc)')

[]